In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from time import time
import os

import sys
sys.path += ['Package/']  

from eder import *
from dcsbm import *
from node_embedding import *

import warnings
warnings.filterwarnings("ignore")

directory_root = '/home/lorenzo/Scrivania/My_projects/DeepWalk/Codes/'

## Test against spectral clustering

In [2]:
def RunSimulation():
    
    _f = lambda x:x**0.5
    
    dfNMI = pd.DataFrame(columns = ['α', 'd32', 'd32ns', 'SC'])
    dft = pd.DataFrame(columns = ['α', 'd32', 'd32ns', 'SC'])
    
    θ = np.random.uniform(3,12, 10000000)**6
    θ = θ/np.mean(θ)
    sq = np.mean(θ**2)

    
    # set the parameters to generate the graph
    N = 30000
    n_clusters = 4
    π = np.ones(n_clusters)/n_clusters

    # average degree and c_out vector
    c = 10
    c_out_v = np.linspace(1, 9.2, 15)

    # label vector
    label = np.concatenate([[j for i in range(int(π[j]*N))] for j in range(n_clusters)])

    θ = np.random.uniform(3, 12, N)**6
    θ = θ/np.mean(θ)
    
    dim = 32
    
    for i, c_out in enumerate(c_out_v):
        
        n = N
        α = (c - c_out)*np.sqrt(sq/c)

        # create the C matrix
        C = matrix_C(c_out, c, 0., π)

        # generate the adjacency matrix
        A, ℓ = adj(C,c, label, θ, True)
        
        tv = []
        NMIv = []

        # Node embedding with degree correction
        t0 = time()
        Φ = NodeEmbedding(A, dim, f_func = _f, n_epochs = 50, η = 0.85, n_prod = 1, verbose = False)
        NMIv.append(computeScore(Φ, ℓ))
        tv.append(time()-t0)
        
        # Node2Vec
        t0 = time()
        Y = Node2VecNS(A, dim, verbose = False)
        tv.append(time() - t0)
        NMIv.append(computeScore(Y, ℓ))


        # Spectral clustering
        t0 = time()
        X = community_detection(A , n_clusters = n_clusters, verbose = False)
        tv.append(time()-t0)
        NMIv.append(computeScore(X, ℓ))
        

        print("[%-25s] %d%%" % ('='*(int((i+1)/len(c_out_v)*25)-1) + '>', (i+1)/(len(c_out_v))*100), end = '\r')

        row = {'α': α, 'd32': NMIv[0], 'd32ns': NMIv[1], 'SC': NMIv[2]}
        dfNMI = dfNMI.append(row, ignore_index = True)
        
        row = {'α': α, 'd32': tv[0], 'd32ns': tv[1], 'SC': tv[2]}
        dft = dft.append(row, ignore_index = True)
        
   
    # save the result
    try:
        nn = (np.max([int(x.split('_')[1]) for x in os.listdir('saved_files/NMI')]))
        dfNMI.to_csv('saved_files/NMI/v_' + str(nn+1) + '_.csv', index = False)

    except:
        dfNMI.to_csv('saved_files/NMI/v_' + str(1) + '_.csv', index = False)
        
        
    try:
        nn = (np.max([int(x.split('_')[1]) for x in os.listdir('saved_files/t')]))
        dft.to_csv('saved_files/t/v_' + str(nn+1) + '_.csv', index = False)

    except:
        dft.to_csv('saved_files/t/v_' + str(1) + '_.csv', index = False)

In [ ]:
n_iterations = 3

for nn in range(n_iterations):
    print('\n' + str(nn+1) + '/' + str(n_iterations) + '\n')
    RunSimulation()


1/3



In [ ]:
dfNMI_list = []
dft_list = []

for filename_NMI, filename_t in zip(os.listdir('saved_files/NMI'), os.listdir('saved_files/t')):
    dfNMI_list.append(pd.read_csv('saved_files/NMI/' + filename_NMI))
    dft_list.append(pd.read_csv('saved_files/t/' + filename_t))
    
dfNMI = pd.concat(dfNMI_list)
dfNMI.α = dfNMI.α.map(lambda x: np.round(x, 2))
dfNMI_m = dfNMI.groupby('α').mean().reset_index()
dfNMI_stdv = dfNMI.groupby('α').std().reset_index()

dft = pd.concat(dft_list)
dft.α = dft.α.map(lambda x: np.round(x, 2))
dft_m = dft.groupby('α').mean().reset_index()
dft_stdv = dft.groupby('α').std().reset_index()

colors = ['firebrick', 'forestgreen', 'navy', 'orange']
markers = ['o', 'D', 'd', '*']
ms = 10
names = list(dfNMI.columns[1:])

fig, ax = plt.subplots(1,2, figsize = (14, 5))

for i, name in enumerate(names):

    ax[0].plot(dfNMI_m.α, dfNMI_m[name], color = colors[i], marker = markers[i], 
                       markersize = ms, fillstyle = 'none', label = name)
    
    ax[0].fill_between(dfNMI_m.α, dfNMI_m[name] - dfNMI_stdv[name]/2, dfNMI_m[name] + dfNMI_stdv[name]/2,
                      color = colors[i], alpha = 0.3)


    ax[1].plot(dft_m.α, np.log10(dft_m[name]), color = colors[i], marker = markers[i], 
                       markersize = ms, fillstyle = 'none')
    
    ax[1].fill_between(dft_m.α, np.log10(dft_m[name] - dft_stdv[name]/2), np.log10(dft_m[name] + dft_stdv[name]/2),
                  color = colors[i], alpha = 0.3)


ax[0].axvline(1, linewidth = 1.5, linestyle = '-.', color = 'k')
ax[0].legend()

plt.show();